In [1]:
from surfacecode.lattice import SquareLattice
from surfacecode.mwpm import ParityCheckMatrix, ErrorCorrection
import numpy as np

In [2]:
WIDTH = 3
HEIGHT = 11
lattice = SquareLattice(WIDTH, HEIGHT)

In [3]:
num_stabilizers = lattice.z_counter + lattice.x_counter
parity = ParityCheckMatrix(num_stabilizers, len(lattice.nodes) - num_stabilizers)

In [4]:
mm = parity._populate(lattice)

In [5]:
correction = ErrorCorrection(parity)

# Cycles

In [6]:
shots = {'01 000001000101000001000001000100000 000001000101000001000001000100000 00000100010100010000010000010000011 000001000101000001000001000100000': 1,
 '00 000001000101000101010101000001000 000001000101000101010101000001000 00000100010100010100000000000100001 000001000101000101000101000001000': 1,
 '11 000101000001000100000001000101000 000101000001000100000001000101000 00010100000100010000010000010100001 000101000001000100000001000101000': 1,
 '11 000100000000000001000000000100000 000100000000000001000000000100000 00010000000000010000000000010000010 000100000000000001000000000100000': 1,
 '10 000001000101000000000000000101000 000001000101000000000000000101000 00000100010100000000000000010100000 000001000101000000000000000101000': 1,
 '10 000001000001000000010001000100000 000001000001000000010001000100000 00000100000100010100010000010000011 000001000001000000000001000100000': 1,
 '11 000001000001000101000000000100000 000001000001000101000000000100000 00000100000100000000000000010000010 000001000001000101000000000100000': 1,
 '11 000001000001000000000100000101000 000001000001000000000100000101000 00000100000100010100010000010100010 000001000001000000000100000101000': 1,
 '01 000101000100000001000000000101000 000101000100000001000000000101000 00010100010000010000010100010100011 000101000100000001000000000101000': 1,
 '10 000100000000000101010001000101000 000100000000000101010001000101000 00010000000000000000010000010100011 000100000000000101000001000101000': 1}



def cycles(key: str, num_stabilizers: int, width: int, height: int):
    cycles = key.split(" ")[1:]
    sep_cycles = []
    for cycle in cycles:
        cycle = cycle[:width * height]
        j = 1
        z_measures = []
        x_measures = []
        for i, measure in enumerate(cycle):
            if i >= width * j:
                j += 1
            if i % 2 == 1:
                if j % 2 == 1:
                    z_measures.append(int(measure))
                else:
                    x_measures.append(int(measure))
                    
        sep_cycle = np.concatenate([np.array(z_measures), np.array(x_measures)])
        sep_cycle = np.array(sep_cycle).reshape(1, -1)
        sep_cycles.append(sep_cycle)

    sep_cycles = np.concatenate(sep_cycles)
    return sep_cycles


In [7]:
a = cycles(list(shots.keys())[0], num_stabilizers, WIDTH, HEIGHT)

In [25]:
a

array([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0]])

In [11]:
lattice.x_counter

10

In [7]:
a = np.zeros((2, 16))
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
a[1][0] = 1
a[1][10] = 1
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [ ]:
D Z D
X D X


In [29]:
a = np.zeros((2, 16))


In [8]:
b = correction.analyze(a)

In [9]:
b

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [10]:
b[:17], b[17:]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [12]:
np.sum(parity.bpcm, axis=0)

array([1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1.,
       1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1.])

In [ ]:
def experimentX(w, h, l, type):
    lattice = SquareLattice(3,11)
    qubit = LQubit(lattice, l[0], l[1], type)
    cycle = SurfaceCodeCycle(lattice)
    num_qubits = w * h

    qc = ConstrainedQuantumCircuit(lattice, w * h)

    qc.add_register(ClassicalRegister(num_qubits))
    qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(num_qubits)))

    qc.add_register(ClassicalRegister(num_qubits + len(qubit.route(l[0],l[1])) // 2))
    qc = qc.compose(qubit.alt_initialize(cycle), list(range(num_qubits)), list(range(num_qubits, 2 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

    qc.add_register(ClassicalRegister(num_qubits))
    qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(2 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 3 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

    qc = qc.compose(qubit.circle_gate())

    qc.add_register(ClassicalRegister(num_qubits))
    qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(3 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 4 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

    qc.add_register(ClassicalRegister(2))
    qc = qc.compose(qubit.alt_measure(), list(range(num_qubits)), list(range(4 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 4 * num_qubits + len(qubit.route(l[0],l[1])) // 2 + 2)))
    return qc